<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 11: Hypothesis Testing Part 1</h1>


<a name='section_11_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.0 Overview</h2>


<h3>Navigation</h3>


<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_1">L11.1 Likelihood Ratio and Hypothesis Testing</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_11_1">L11.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_2">L11.2 Hypothesis Test Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_11_2">L11.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_3">L11.3 A More Realistic Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_11_3">L11.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_4">L11.4 Probability of Fit</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_11_4">L11.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_11_5">L11.5 The t-test Statistic</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_11_5">L11.5 Exercises</a></td>
    </tr>
</table>



In [ ]:
#>>>RUN: L11.0-runcell00

!pip install lmfit

In [ ]:
#>>>RUN: L11.0-runcell01

import numpy as np                #https://numpy.org/doc/stable/ 
import lmfit                      #https://lmfit.github.io/lmfit-py/ 
import matplotlib.pyplot as plt   #https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
from scipy import stats           #https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
#>>>RUN: L11.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title



<a name='section_11_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.1 Likelihood Ratio and Hypothesis Testing</h2>  

| [Top](#section_11_0) | [Previous Section](#section_11_0) | [Exercises](#exercises_11_1) | [Next Section](#section_11_2) |


In [ ]:
#>>>RUN: L11.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_06.html', width=970, height=550)

In [ ]:
#>>>RUN: L11.1-runcell01

np.random.seed(5)

testSamplesA = np.random.normal(0  ,1, 50)
testSamplesB = np.random.normal(0.2,1, 50)

def plotHist(iSamples,iLabel,bin_edges=None):
    if bin_edges is None:
        y, bin_edges = np.histogram(iSamples, bins=5)
    else:
        y, bin_edges = np.histogram(iSamples, bins=bin_edges)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm=len(iSamples)
    plt.errorbar(bin_centers,y/norm,yerr=y**0.5/norm,fmt=".",label=iLabel)
    return bin_edges

bin_edges=plotHist(testSamplesA,"p$_{a}$(x)")
bin_edges=plotHist(testSamplesB,"p$_{b}$(x)",bin_edges)
plt.xlabel("x")
plt.ylabel("p")
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L11.1-runcell02

#Now let's compute the log-likelihood values of our samples
#ulimately, we will use this function to see if Wilks' theorem is true (in the next section)

#Let's define the log-likelihood in the simplest way possible (assume we don't know math)
#note we will use -1 to compute these on the fly
def loglikelihoodGaus(isample,mu0=-1,sigma0=-1):
    n=len(isample)
    if mu0 == -1:
        mu0=isample.mean()
    if sigma0 == -1:
        sigmahat2=(isample-mu0)**2/len(isample)
        sigmahat2=sum(sigmahat2)
    val=-n/2*(np.log(2*np.pi*sigmahat2)+1)
    return val

hypoth=loglikelihoodGaus(testSamplesA,0) #compute with a fixed mean of 0
null  =loglikelihoodGaus(testSamplesB) #compute with mean floating
print(hypoth-null)

<a name='exercises_11_1'></a>     

| [Top](#section_11_0) | [Restart Section](#section_11_1) | [Next Section](#section_11_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.1.1</span>

Which of the following best describes the use of the likelihood ratio in statistical inference?

A) To compare the fit of two models to a given set of data\
B) To calculate the probability of observing a particular value of a parameter\
C) To determine the significance of a test statistic\
D) To estimate the distribution of a random variable

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.1.2</span>

The first hypothesis test was developed by a statistician (Fisher) when he was serving tea to Muriel Bristol and her fiancee William Roach. Muriel claimed that she could tell whether milk was poured into a tea cup before the tea was poured in or after it was poured in.

To test this, the statistician made 8 cups of tea, with 4 of those cups having milk before the tea was poured, and 4 of those cups having milk after the tea was poured. 

What is the null Hypothesis in this test? Choose the best answer from the following:

A) Muriel will select all cups correctly.\
B) Muriel will not have any correct selections.\
C) Muriel will just guess randomly, and so will have some correct and some incorrect selections.\
D) Muriel will have exactly 1/2 correct selections and 1/2 incorrect selections.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.1.3</span>

Muriel apparently guessed all 4 cups correctly that had milk poured in before (and likewise the other 4 that had it poured after). What is the probability that she guessed randomly and just got lucky? Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L11.1.3
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

from scipy.special import comb

#Hint: use comb(n,k), where
#comb(n,k) is the number of combinations of n with k chosen


<a name='section_11_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.2 Hypothesis Test Example</h2>  

| [Top](#section_11_0) | [Previous Section](#section_11_1) | [Exercises](#exercises_11_2) | [Next Section](#section_11_3) |


In [ ]:
#>>>RUN: L11.2-runcell01

np.random.seed(41)

def sampleGaus(iMean,iNToy=10000):
    ntoys=iNToy
    deltaLL=[]
    for i0 in range(ntoys):
        testsamples1 = np.random.normal(iMean,1, 50)
        hypoth=loglikelihoodGaus(testsamples1,0) #compute with a fixed mean of 0
        null  =loglikelihoodGaus(testsamples1) #compute with mean floating
        val=hypoth-null
        deltaLL.append(-2*val)
    return deltaLL

deltaLL1 = sampleGaus(0)
deltaLL2 = sampleGaus(0.2)
    
y1, bin_edges = np.histogram(deltaLL1, bins=20)
y2, bin_edges = np.histogram(deltaLL2, bins=bin_edges)

#Now plot a chi2 with 1 dof
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

#And Draw
fig, ax = plt.subplots(figsize=(9,6))
plt.style.use('fast')
plt.errorbar(bin_centers,y1/len(deltaLL1),yerr=y1**0.5/len(deltaLL1),marker='.',drawstyle = 'steps-mid',label='$\mu=0$')
plt.errorbar(bin_centers,y2/len(deltaLL2),yerr=y2**0.5/len(deltaLL2),marker='.',drawstyle = 'steps-mid',label='$\mu=0.2$')
plt.xlabel("$\Delta$ log-likelihood")
plt.ylabel("Toys")
ax.set_yscale('log')
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L11.2-runcell02

np.random.seed(41)

#Now plot a chi2 with 1 dof
chi2 = []
for x in bin_centers:#range(len(bin_centers)):
    chi2val=stats.chi2.pdf(x, 1)
    chi2.append(chi2val)

#And Draw
fig, ax = plt.subplots(figsize=(9,6))
plt.style.use('fast')
plt.errorbar(bin_centers,y1/len(deltaLL1),yerr=y1**0.5/len(deltaLL1),marker='.',drawstyle = 'steps-mid',label='$\mu=0$')
plt.errorbar(bin_centers,y2/len(deltaLL2),yerr=y2**0.5/len(deltaLL2),marker='.',drawstyle = 'steps-mid',label='$\mu=0.2$')
plt.plot(bin_centers,chi2,label='$\chi^{2}_{1}(x)$')
plt.xlabel("$\Delta$ log-likelihood")
plt.ylabel("Toys")
ax.set_yscale('log')
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L11.2-runcell03

#let's try to understand the deltaLL values 
#of the distributions we defined earlier

np.random.seed(5)

testSamplesA = np.random.normal(0  ,1, 50)
testSamplesB = np.random.normal(0.2,1, 50)

hypoth=loglikelihoodGaus(testSamplesA,0) #compute with a fixed mean of 0
null  =loglikelihoodGaus(testSamplesA) #compute with mean floating
deltaLL = -2.*(hypoth-null)
print('deltaLL testSamplesA:', deltaLL)

hypoth=loglikelihoodGaus(testSamplesB,0) #compute with a fixed mean of 0
null  =loglikelihoodGaus(testSamplesB) #compute with mean floating
deltaLL = -2.*(hypoth-null)
print('deltaLL testSamplesB:', deltaLL)

<a name='exercises_11_2'></a>     

| [Top](#section_11_0) | [Restart Section](#section_11_2) | [Next Section](#section_11_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.2.1</span>

In the first plot in this section, why is the orange histogram (calculated from a distribution with `mu=0`) above the blue histogram (calculated from a distribution with `mu=0.2`)?

A) The orange data is well described by the model with `mu=0.2`\
B) There are systematic errors or biases in the orange data\
C) The model being tested is overfitting the orange data\
D) The orange data is not well described by the model with `mu=0`

<a name='section_11_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.3 A More Realistic Example</h2>  

| [Top](#section_11_0) | [Previous Section](#section_11_2) | [Exercises](#exercises_11_3) | [Next Section](#section_11_4) |


In [ ]:
#>>>RUN: L11.3-runcell01

np.random.seed(42)

#First let's make a data sample and fit it
bkg = np.random.uniform(0,200, 300)
sig = np.random.normal (100,15,200) #choose different options for the number of signal events
#sig = np.random.normal (100,15,80)
data = np.append(sig,bkg)
y, bin_edges = np.histogram(data, bins=20)
ys, bin_edges = np.histogram(sig, bins=bin_edges)
yb, bin_edges = np.histogram(bkg, bins=bin_edges)

bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
plt.errorbar(bin_centers, y,yerr= y**0.5,marker='.',linestyle = 'None', color = 'black')
#plt.errorbar(bin_centers,yb,yerr=ys**0.5,marker='.',linestyle = '-', color = 'red',label='bkg')
#plt.errorbar(bin_centers,ys,yerr=yb**0.5,marker='.',linestyle = '-', color = 'blue',label='signal')
#plt.legend()
plt.xlabel("x")
plt.ylabel("N")
plt.show()


In [ ]:
#>>>RUN: L11.3-runcell02

from scipy.stats import norm
import lmfit

def fNull(params,x):
    val = norm.pdf(x,params["mu"],params["sigma"])
    return params["amp"]*val + params["c"]

def fAlt(params,x):
    return params["c"]*(x+1)/(x+1) #hack to output array

def binnedLikelihood(params, x, ydata, weights, func):
    y_model= func(params,x)
    residarr = np.sqrt(((y_model - ydata)/(2*weights))**2 + np.log(2*np.pi*weights**2))
    return residarr

def fitData(iX,iY,iFunc,iPlot=False):
    params = lmfit.Parameters()
    params.add('c',    value=1,min=0,max=np.inf)
    params.add('mu',   value=100,min=0,max=200)
    params.add('sigma',value=10,min=0,max=200)
    params.add('amp',  value=20,min=0,max=np.inf)
    result = lmfit.minimize(binnedLikelihood, params, args=(iX,iY,(iY**0.5),iFunc))
    #lmfit.report_fit(result)
    if iPlot:
        #Now we plot it. 
        plt.errorbar(iX, iY,np.sqrt(iY), lw=2,fmt=".k", capsize=0)
        plt.plot(iX,iFunc(result.params,iX))
        plt.xlabel("x")
        plt.ylabel("N")
        plt.show()
    return result.residual

def deltaNLL(x,y,iPlot=False):
    LLNull=fitData(x,y,fNull,iPlot)
    LLAlt=fitData(x,y,fAlt,iPlot)
    return 2.*(np.sum(LLAlt*LLAlt)-np.sum(LLNull*LLNull))

x=bin_centers
NLL = deltaNLL(x,y,True)
print("Delta negative log-likelihood:",NLL)

In [ ]:
#>>>RUN: L11.3-runcell03

def toyNLL(iNEvents,iNToys,iBin_Edges):
    deltaNLLArr=np.array([])
    for i0 in range(iNToys):
        if i0 % 50 == 0: #change to output message more frequently, e.g., i0 % 10 == 0
            print("Toy:",i0," of ",iNToys)
        bkg = np.random.uniform(0,200, iNEvents)
        y, bin_edges = np.histogram(bkg, bins=iBin_Edges)
        x = 0.5*(bin_edges[1:] + bin_edges[:-1])
        nll = deltaNLL(x,y)
        deltaNLLArr=np.append(deltaNLLArr,nll)
    return deltaNLLArr

NLL1 = toyNLL(300,100,bin_edges) #adjust the number of runs as needed
ll1, ll_bin_edges = np.histogram(NLL1, bins=20,density=True)
ll_bin_centers = 0.5*(ll_bin_edges[1:] + ll_bin_edges[:-1])
chi21 = np.array([])
for x in ll_bin_centers:#range(len(bin_centers)):
    chi2val=stats.chi2.pdf(x, 3) #len is to normalize
    chi21 = np.append(chi21,chi2val)

fig, ax = plt.subplots(figsize=(9,6))
plt.plot(ll_bin_centers,ll1,drawstyle = 'steps-mid',label='100 signal toys')
plt.plot(ll_bin_centers,chi21,label='$\chi^{2}_{3}$')
ax.axvline(x=NLL,linewidth=3,c='b',label="observed $\Delta$ LL")
print("Significance with 3 floating",stats.chi2.cdf(NLL,3))
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
#>>>RUN: L11.3-runcell04

#NOW WE FIX MU IN OUR FITTING PROCEDURE: params["mu"].vary = False

def fitData(iX,iY,iFunc,iPlot=False):
    params = lmfit.Parameters()
    params.add('c',    value=1,min=0,max=np.inf)
    params.add('mu',   value=100,min=0,max=200) #try changing the value of mu, when we are leaving it fixed
    params.add('sigma',value=10,min=0,max=200)
    params.add('amp',  value=20,min=0,max=np.inf)
    params["mu"].vary = False
    result = lmfit.minimize(binnedLikelihood, params, args=(iX,iY,(iY**0.5),iFunc))
    #lmfit.report_fit(result)
    if iPlot:
        #Now we plot it. 
        plt.errorbar(iX, iY,np.sqrt(iY), lw=2,fmt=".k", capsize=0)
        plt.plot(iX,iFunc(result.params,iX))
        plt.xlabel("x")
        plt.ylabel("N")
        plt.show()
    return result.residual

def deltaNLL(x,y,iPlot=False):
    LLNull=fitData(x,y,fNull,iPlot)
    LLAlt=fitData(x,y,fAlt,iPlot)
    return 2.*(np.sum(LLAlt*LLAlt)-np.sum(LLNull*LLNull))

x=bin_centers
NLL = deltaNLL(x,y,True)
print("Delta negative log-likelihood:",NLL)

<a name='exercises_11_3'></a>     

| [Top](#section_11_0) | [Restart Section](#section_11_3) | [Next Section](#section_11_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.3.1</span>

How many degrees of freedom should we use when calculating the chi-square value associated with the delta log-likelihood, for the model where `mu` is a fixed parameter? Enter your answer as an integer.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.3.2</span>

Scan the mean of the bump, and make a plot of delta log-likelihood vs mean. What are the features of this plot?

Specifically, complete the following code, which varies the parameter $\mu$ over the range `[60-140]` and generates a plot of `NLL` vs. $\mu$. Enter the value of $\mu$ that maximizes `NLL` as a number with precision 1e-1.

In [ ]:
#>>>EXERCISE: L11.3.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def fitData_vary_mu(iX,iY,iFunc,imu,iPlot=False):
    params = lmfit.Parameters()
    params.add('c',    value=1,min=0,max=np.inf)
    params.add('mu',   value=imu,min=0,max=200) #try changing the value of mu, when we are leaving it fixed
    params.add('sigma',value=10,min=0,max=200)
    params.add('amp',  value=20,min=0,max=np.inf)
    params["mu"].vary = False
    result = lmfit.minimize(binnedLikelihood, params, args=(iX,iY,(iY**0.5),iFunc))
    #lmfit.report_fit(result)
    if iPlot:
        #Now we plot it. 
        plt.errorbar(iX, iY,np.sqrt(iY), lw=2,fmt=".k", capsize=0)
        plt.plot(iX,iFunc(result.params,iX))
        plt.xlabel("x")
        plt.ylabel("N")
        plt.show()
    return result.residual

def deltaNLL_vary_mu(x,y,imu,iPlot=False):
    LLNull=fitData_vary_mu(x,y,fNull,imu,iPlot)
    LLAlt=fitData_vary_mu(x,y,fAlt,imu,iPlot)
    return 2.*(np.sum(LLAlt*LLAlt)-np.sum(LLNull*LLNull))


n_points = 100  # number of points to sample between 60 and 140
imu_values = pass #YOUR CODE HERE (sample n_points between 60 and 140)

# array to store NLL values
nll_values = pass #YOUR CODE HERE

# loop over imu values and calculate deltaNLL for each value
# store in nll_values
# e.g., for i, imu in ...
    

# find the value of imu that minimizes NLL (i.e., maximizes deltaNLL)
imu_max = pass #YOUR CODE HERE
print("mu value that maximizes deltaNLL:", imu_max)
    
# plot NLL vs imu
plt.plot(imu_values, nll_values)
plt.axvline(imu_max, color='r', linestyle='--')
plt.xlabel("mu")
plt.ylabel("NLL")
plt.show()


<a name='section_11_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.4 Probability of Fit</h2>     

| [Top](#section_11_0) | [Previous Section](#section_11_3) | [Exercises](#exercises_11_4) | [Next Section](#section_11_5) |


In [ ]:
#>>>RUN: L11.4-runcell01

#HERE WE AGAIN DEFINE THE FUNCTIONS

def fitData(iX,iY,iFunc,iPlot=False):
    params = lmfit.Parameters()
    params.add('c',    value=1,min=0,max=np.inf)
    params.add('mu',   value=100,min=0,max=200) #try changing the value of mu, when we are leaving it fixed
    params.add('sigma',value=10,min=0,max=200)
    params.add('amp',  value=20,min=0,max=np.inf)
    params["mu"].vary = False
    result = lmfit.minimize(binnedLikelihood, params, args=(iX,iY,(iY**0.5),iFunc))
    #lmfit.report_fit(result)
    if iPlot:
        #Now we plot it. 
        plt.errorbar(iX, iY,np.sqrt(iY), lw=2,fmt=".k", capsize=0)
        plt.plot(iX,iFunc(result.params,iX))
        plt.xlabel("x")
        plt.ylabel("N")
        plt.show()
    return result.residual

def deltaNLL(x,y,iPlot=False):
    LLNull=fitData(x,y,fNull,iPlot)
    LLAlt=fitData(x,y,fAlt,iPlot)
    return 2.*(np.sum(LLAlt*LLAlt)-np.sum(LLNull*LLNull))


x=bin_centers
NLL = deltaNLL(x,y,True)
print("Delta negative Log Likelihood:",NLL)

probability = 1-stats.chi2.cdf(NLL, 2)
print("Our Probability of this happening",probability)

In [ ]:
#>>>RUN: L11.4-runcell02

np.random.seed(42)

#First let's make a data sample and fit it
bkg = np.random.uniform(0,200, 300)
sig = np.random.normal (100,15,10) #choose different options for the number of signal events
data = np.append(sig,bkg)
y, bin_edges = np.histogram(data, bins=20)
ys, bin_edges = np.histogram(sig, bins=bin_edges)
yb, bin_edges = np.histogram(bkg, bins=bin_edges)

bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
plt.errorbar(bin_centers, y,yerr= y**0.5,marker='.',linestyle = 'None', color = 'black')
#plt.errorbar(bin_centers,yb,yerr=ys**0.5,marker='.',linestyle = '-', color = 'red',label='bkg')
#plt.errorbar(bin_centers,ys,yerr=yb**0.5,marker='.',linestyle = '-', color = 'blue',label='signal')
#plt.legend()
plt.xlabel("x")
plt.ylabel("N")
plt.show()


In [ ]:
#>>>RUN: L11.4-runcell03

x=bin_centers
NLL = deltaNLL(x,y,True)
print("Delta negative log-likelihood:",NLL)


probability = 1-stats.chi2.cdf(NLL, 2)
print("Our Probability of this happening by chance",probability)

In [ ]:
#>>>RUN: L11.4-runcell04

np.random.seed(42)

#First let's make a data sample and fit it
bkg = np.random.uniform(0,200, 300)
sig = np.random.normal (100,15,30) #choose different options for the number of signal events
data = np.append(sig,bkg)
y, bin_edges = np.histogram(data, bins=20)
ys, bin_edges = np.histogram(sig, bins=bin_edges)
yb, bin_edges = np.histogram(bkg, bins=bin_edges)

bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
plt.errorbar(bin_centers, y,yerr= y**0.5,marker='.',linestyle = 'None', color = 'black')
#plt.errorbar(bin_centers,yb,yerr=ys**0.5,marker='.',linestyle = '-', color = 'red',label='bkg')
#plt.errorbar(bin_centers,ys,yerr=yb**0.5,marker='.',linestyle = '-', color = 'blue',label='signal')
#plt.legend()
plt.xlabel("x")
plt.ylabel("N")
plt.show()


In [ ]:
#>>>RUN: L11.4-runcell05

x=bin_centers
NLL = deltaNLL(x,y,True)
print("Delta negative log-likelihood:",NLL)

probability = 1-stats.chi2.cdf(NLL, 2)
print("Our Probability of this happening by chance",probability)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.4.1</span>

Consider your plot from Ex-11.3.2, where you scanned over a range of values for the mean of the bump and calculated the delta log-likelihood.

Now, define a function to change the y-axis to a pvalue. Complete the following code to define the function `get_prob_NLL(NLL)`, then run the cell to plot the probability as a function of $\mu$. Note, you will need to have run your solution for Ex-11.3.2 to make this plot.

In [ ]:
#>>>EXERCISE: L11.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


def get_prob_NLL(NLL):
    return #YOUR CODE HERE


#process the NLL array to reassign any outliers
def reassgin_outliers(arr):
    for i in range(1, len(arr)-1):
        if abs(arr[i] - arr[i-1]) / arr[i-1] > 0.5:
            arr[i] = (arr[i-1]+arr[i+1])/2.
    return arr


nll_values_processed = reassgin_outliers(nll_values)

# plot prob vs imu
plt.plot(imu_values, get_prob_NLL(nll_values_processed))
plt.axvline(imu_max, color='r', linestyle='--')
plt.xlabel("mu")
plt.ylabel("Probability")
plt.show()


<a name='section_11_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.5 The t-test Statistic</h2>     

| [Top](#section_11_0) | [Previous Section](#section_11_4) | [Exercises](#exercises_11_5) |


In [ ]:
#>>>RUN: L11.5-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L12/slides_L12_01.html', width=970, height=550)

In [ ]:
#>>>RUN: L11.5-runcell01

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

#now let's test if our observation is consistent with a given value of mu
def ttest(isamples,iMu=0):
    t = (isamples.mean()-iMu)/(isamples.std()/np.sqrt(len(isamples)))
    p = 1 - stats.t.cdf(t,df=len(isamples)-1)
    return p

xvals=[]
pvalues1=[]
pvalues2=[]
pvalues3=[]
for i0 in range(50):
    testsamples1 = np.random.normal(0.2,1, i0)
    pvalue = ttest(testsamples1)
    pvalues1.append(pvalue)
    testsamples2 = np.random.normal(1,1, i0)
    pvalue = ttest(testsamples2)
    pvalues2.append(pvalue)
    testsamples3 = np.random.normal(2,1, i0)
    pvalue = ttest(testsamples3)
    pvalues3.append(pvalue)
    xvals.append(i0)

fig, ax = plt.subplots(figsize=(9,6))
plt.style.use('fast')
ax.plot(xvals,pvalues1,label="$\mu=0.2$")
ax.plot(xvals,pvalues2,label="$\mu=1.0$")
ax.plot(xvals,pvalues3,label="$\mu=2.0$")
ax.set_xlabel('N')
ax.set_ylabel('p-value')
ax.set_yscale('log')
plt.legend()
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.5.1</span>

Complete the code below, to compare the t-test vs. $\chi^{2}$ statistic for measuring the similarity of two data sets. These functions should return the p-values for the two tests. Which test is more sensitive?

In [ ]:
#>>>EXERCISE: L11.5.1

np.random.seed(42)

testSamplesA = np.random.normal(0  ,1, 50)
testSamplesB = np.random.normal(0.2,1, 50)

def chi2test(iSampleB,iSampleA,nbins=5):
    countA, binsA, ignored = plt.hist(iSampleA, nbins,      density=True ,alpha=0.5,label='a')
    countB, binsB, ignored = plt.hist(iSampleB, bins=binsA, density=True,alpha=0.5,label='b')
    chi2=0
    ndof=0
    for i0 in range(len(binsA)-1):
        if countB[i0] > 0 and countA[i0] > 0:
            unc2 = countA[i0]/len(iSampleA) +  countB[i0]/len(iSampleB)
            chi2+=(countB[i0]-countA[i0])**2/unc2
            ndof=ndof+1
    #now we rely on the chi2 distribution to get the probability
    p= #YOUR CODE HERE
    return p

def ttest_comp(iSamplesA,iSamplesB):
    s2 = (len(iSamplesA)-1)*iSamplesA.std()**2 + (len(iSamplesB)-1)*iSamplesB.std()**2
    s2 = s2/(len(iSamplesA)+ len(iSamplesB) - 2)
    s  = np.sqrt(s2)*(1/len(iSamplesA) + 1/len(iSamplesB))
    t  = ### YOUR CODE HERE
    p  = stats.t.cdf(t,df=len(iSamplesA)+len(iSamplesB)-2)
    return p

print("Chi2",chi2test(testSamplesA,testSamplesB))
print("t-test",ttest_comp(testSamplesA,testSamplesB))